In [1]:
import numpy as np
import seaborn as sns
import gc

In [2]:
init_state = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]
])

In [3]:
class Game:
    def __init__(self, state, FIRST=1):
        self.state = state
        self.empty = self.make_empty(state)
        self.first_player = FIRST
        
    def make_empty(self, state):
        emp = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    emp.append(3*i + j)
        
        return emp
    
    def is_lose(self):
        a = self.next_opp()
        
        for i in range(3):
            if self.state[i][0] == self.state[i][1] == self.state[i][2] == a:
                return True
            elif self.state[0][i] == self.state[1][i] == self.state[2][i] == a:
                return True
        if self.state[0][0] == self.state[1][1] == self.state[2][2] == a:
            return True
        if self.state[0][2] == self.state[1][1] == self.state[2][0] == a:
            return True
        return 0
    
#     def is_win(self):       
#         a = self.next_opp()
#         for i in range(3):
#             if self.state[i][0] == self.state[i][1] == self.state[i][2] == a:
#                 return True
#             elif self.state[0][i] == self.state[1][i] == self.state[2][i] == a:
#                 return True
#         if self.state[0][0] == self.state[1][1] == self.state[2][2] == a:
#             return True
#         if self.state[0][2] == self.state[1][1] == self.state[2][0] == a:
#             return True
        
#         return False
    
    def is_draw(self):
        a = self.next_opp()
        if self.is_lose():
            return 0
        if np.all(self.state):
            return 1
        else:
            return 0
        
    def is_done(self):
        if self.is_lose() or self.is_draw():
            return 1
        else:
            return 0
        
        
    def update(self, target):
        state = self.state.copy()
        x, y = target//3, target%3
        a = self.next_opp()
        state[x][y] = a
        return Game(state)
    
    
    def next_opp(self):
        a = b = 0
        for i in range(len(self.state)):
            for j in range(len(self.state)):
                if self.state[i][j] == self.first_player:
                    a += 1
                elif self.state[i][j] != 0:
                    b += 1
                    
        if a == b:
            return self.first_player
        else:
            return 2 + min(0, 1-self.first_player)

In [ ]:
# def mcs(game):
#     n = 10
#     values = [0] * len(game.empty)
    
#     for i, a in enumerate(game.empty):
#         for _ in range(n):
#             values[i] += -playout(game.update())
    

In [4]:
class Random:       
    def action(self, game):
        return np.random.choice(game.empty)

In [5]:
class MCS:
    def __init__(self, n=20):
        self.n = n
    
    def playout(self, game):
        if game.is_lose():
            return -1
        
        if game.is_draw():
            return 0
        
        
#         if game.is_win(self.status):
#             return 1
        
        return -self.playout(game.update(np.random.choice(game.empty)))
    
    
    def action(self, game):
        values = [0] * len(game.empty)

        for i, a in enumerate(game.empty):
            for _ in range(self.n):
                g = game.update(a)
                values[i] += self.playout(g)
                
        return game.empty[np.argmax(values)]


In [6]:
gc.collect()

60

In [7]:
def play(game, m1, m2):
    global score
    while 1:
        a1 = m1.action(game)
        game = game.update(a1)
#         print(game.state)
        if game.is_lose():
            score[0] += 1
#             print(game.state)
            return 
        elif game.is_draw():
            score[2] += 1
#             print(game.state)
            return 

        a2 = m2.action(game)
        game = game.update(a2)
#         print(game.state)
        if game.is_lose():
            score[1] += 1
#             print(game.state)
            return 
        elif game.is_draw():
            score[2] += 1
#             print(game.state)
            return 
        

In [8]:
game = Game(init_state)
m1 = Random()
m2 = MCS()

In [9]:
%%time
score = [0, 0, 0]
for _ in range(1000):
    play(game, m1, m2)

print(score)
score = [0, 0, 0]
for _ in range(1000):
    play(game, m2, m1)
print(score)

[861, 52, 87]
[8, 975, 17]
Wall time: 5min 40s


In [ ]:
# sns.barplot(x = [1, 2], y = score[:2])

In [ ]:
# v1: 710 vs 1208
# v2: 113 vs 1771 // 113 vs 814
# v3: 95 vs 1801 // 89 vs 829
# v4: 60 vs 1836 // complete
## 이정도면 완성형
